# Navigating by Falling Stars: Monetary Policy with Fiscally Driven Natural Rates
## Calibration and calculation of the initial and final steady state

This notebook does the following:
1. It calibrates the model and computes the initial steady state. A dictionary with the values of the calibration and this initial steady state are saved into `ss/ss_hank_ini.json`.
2. It calibrates a new steady state with debt that is higher (10% of initial GDP higher). A dictionary with the values of this new steady state are saved into `ss/ss_hank_end.json`.
3. It produces Table 4 in the paper. This table is exported to `results/table_4.tex`.
5. It calculates an additional steady state in which the central bank does change the intercept in its Taylor rule. A dictionary with the values of this steady state are saved into `ss/ss_hank_end_no_reaction.json`.

### Requirements
This code imports the model from the file `model.py` and various functions from `utils.py`. The code is otherwise self-contained.

In [1]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sequence_jacobian as sj

from model import hank_lb, hh_ext, income
from utils import broyden_solver


In [2]:
# This function calculates intertemporal MPCs
def compute_iMPCs(model, ss, T=300):
    J_household = model.jacobian(ss, inputs=["Transfer"],  T=T)
    M = J_household['C']['Transfer']
    return M
 
def compute_on_impact_MPC(model, ss):
    M = compute_iMPCs(model, ss)
    return (M[0, 0], np.sum(M[:4, 0]))

def compute_htm(ss):
    data = ss.internals['hh']
    D = data['D']
    htm_share = D[:, 0, :].sum()
    whtm_share = D[:, 0, 1:].sum()
    phtm_share = D[:, 0, 0].sum()
    return htm_share, whtm_share, phtm_share

In [3]:
def hank_ss(r_guess=0.007837279739129012, chi1_guess=37.43463865033339, chi0_guess=3.7824953551757288,
            beta=0.988, tot_wealth=14.0, K=9.0, delta=0.02, kappap=0.1, delta_lb=0.95, pi=1.02**(1/4)-1,
            muw=1.1, Bg=2.8, G=0.2, eis=1.0, frisch=1.0, chi2=2, chi3=0.0, epsI=4, rb=1.01**(1/4)-1, kappaw=0.1,
            phi=1.25, nZ=3, nB=50, nA=50, nK=50, bmax=20000, bmin=0.0, amin=0.0, amax=20000, kmax=1, rho_z=0.966, sigma_z=0.92,
            tax=0.3, mup=1.1, htm_target=0.41, noisy=True):
    """Solve steady state of full GE model"""

    # Household parameters
    hh_params = {'amin': amin, 'bmin': bmin, 'bmax': bmax, 'amax': amax, 'kmax': kmax, 'nB': nB, 'nA': nA,
                 'nK': nK,'nZ': nZ, 'rho_z': rho_z, 'sigma_z': sigma_z }
    
    # set up grid
    b_grid = sj.grids.asset_grid(amin=bmin,amax=bmax, n=nB)
    a_grid = sj.grids.asset_grid(amin=amin,amax=amax, n=nA)
    k_grid = sj.grids.asset_grid(amin=amin,amax=kmax, n=nK)[::-1].copy()
    # e_grid, _, Pi = utils.markov_rouwenhorst(rho=rho_z, sigma=sigma_z, N=nZ)
    e_grid, _, Pi = sj.grids.markov_rouwenhorst(rho=rho_z, sigma=sigma_z, N=nZ)
    
    # solve analytically what we can
    I = delta * K
    
    # residual function
    def res(x):
        r_loc, chi1_loc, chi0_loc = x
        
        mc = 1 / mup
        tau_d = 1 - (tot_wealth - Bg) / ( (1-mc) / r_loc + K ) 
        alpha = (r_loc + delta) * K / mc
        Z = K ** (-alpha)
        w = (1 - alpha) * mc
        div = (1 - w - I) * (1-tau_d)
        T_d =  (1 - w - I) * tau_d
        
        Transfer = tax * w + T_d - (G + rb * Bg)
        T = tax * w  + T_d
        p = div / r_loc

        ra = r_loc

        # figure out initializer
        z_grid = income(e_grid, tax, w, 1, Transfer)
        Va = (0.6 + 1.1 * b_grid[:, np.newaxis] + a_grid) ** (-1 / eis) * np.ones((z_grid.shape[0], 1, 1))
        Vb = (0.5 + b_grid[:, np.newaxis] + 1.2 * a_grid) ** (-1 / eis) * np.ones((z_grid.shape[0], 1, 1))
        
        if r_loc < rb or chi1_loc < 0.05 or chi0_loc < 0.01 :
            raise ValueError('Clearly invalid inputs')
        d = {**hh_params, 'Vb': Vb, 'Va': Va, 'N': 1, 'tax': tax, 'Transfer': Transfer, 'T': T,
             'w': w, 'rb': rb, 'ra': ra, 'beta': beta, 'eis': eis,
             'chi0': chi0_loc,'chi1': chi1_loc,'chi2': chi2,'chi3': chi3}
        out = hh_ext.steady_state(d)
        
        htm_share, whtm, phtm = compute_htm(out)

        asset_mkt = out['A'] + out['B'] - p - Bg
        return np.array([asset_mkt, out['B'] - Bg, htm_share - htm_target])

    # solve for the three variables
    (ra, chi1, chi0), _ = broyden_solver(res, np.array([r_guess, chi1_guess, chi0_guess]), tol=1E-5, noisy=noisy)

    # other things of interest
    mc = 1 / mup
    tau_d = 1 - (tot_wealth - Bg) / ((1-mc)/ra + K) 
    alpha = (ra + delta) * K / mc
    Z = K ** (-alpha)
    w = (1 - alpha) * mc
    div = (1 - w - I) * (1-tau_d)
    T_d =  (1 - w - I) * tau_d
    Transfer = tax * w + T_d - (G + rb * Bg)
    T = tax * w  + T_d
    p = div / ra

    # extra evaluation to report variables
    dd = {**hh_params, 'N': 1, 'tax': tax, 'Transfer': Transfer, 'T': T,
          'w': w, 'rb': rb, 'ra': ra, 'beta': beta, 'eis': eis, 
          'chi0': chi0, 'chi1': chi1, 'chi2': chi2, 'chi3': chi3}
    ss = hh_ext.steady_state(dd)
    
    # Disutility of working
    vphi = (1 - tax) * w * ss['UCE'] / muw
    
    # Long bonds
    Q_lb = 1 / ((1+rb)*(1+pi) - delta_lb)
    
    pshare = p / ss['A']
    
    # calculate aggregate adjustment cost and check Walras's law
    Chi = ss['CHI']
    goods_mkt = ss['C'] + I + G + Chi - 1
    print("Goods market error:", np.abs(goods_mkt))
    
    ss.update({'pi': pi, 'piw': pi, 'pi_ss': pi, 'pibar':pi, 'Q': 1, 'Y': 1, 'N': 1, 'mc': mc, 
               'K': K, 'Z': Z, 'I': I, 'w': w, 'tax': tax,
               'Transfer': Transfer, 'T': T, 'div': div, 'p': p, 'ra_e': ra, 'Bg': Bg, 'Bgbar': Bg, 
               'G': G, 'Gbar': G, 'Chi': Chi, 'goods_mkt': goods_mkt, 'phi': phi,
               'beta': beta, 'vphi': vphi, 'alpha': alpha, 'delta': delta, 'mup': mup, 'muw': muw,
               'frisch': frisch, 'epsI': epsI, 'a_grid': a_grid, 'b_grid': b_grid, 'e_grid': e_grid,
               'k_grid': k_grid, 'Pi': Pi, 'kappap': kappap, 'kappaw': kappaw, 'pshare': pshare,
               'psip': 0, 'psiw': 0, 'k_adjust': 0, 'tau_d': tau_d, 'Q_lb': Q_lb, 'delta_lb': delta_lb,
               'phi_G': 0.1, 'rho_i': 0.8, 'rb_e': rb, 'rbar': rb, 'i': (1+rb)*(1+pi) - 1})

    return ss

In [4]:
c = hank_ss(noisy=True)
c = {**c,  }

On iteration 0
x = 0.008, 37.435, 3.782
y = 0.005, -0.006, 0.001


On iteration 1
x = 0.008, 37.570, 3.782
y = 0.001, -0.000, 0.000


On iteration 2
x = 0.008, 37.570, 3.782
y = 0.000, -0.000, -0.000


Goods market error: 6.476926284193496e-08


In [5]:
# Compute the steady state of the full model
ss = hank_lb.steady_state(c, dissolve=['G_rule', 'taylor_smooth'])
print(f"Liquid assets: {ss['B']: 0.2f}")
print(f"Illiquid assets: {ss['A']: 0.2f}")
print(f"Asset market clearing: {ss['asset_mkt']: 0.2e}")
print(f"Bond market clearing: {ss['bond_mkt']: 0.2e}")
print(f"Goods market clearing (untargeted): {ss['goods_mkt']: 0.2e}")

def is_residual(name):
    if '_res' in name:
        return True
    elif '_mkt' in name:
        return True
    else:
        return False

for k, v in ss.toplevel.items():
    if is_residual(k):
        print(f"{k}: {v:.6f}")
        
M = compute_iMPCs(hank_lb, ss, T=300)
print(f"The quarterly MPC in the {hank_lb.name} is {np.sum(M[0, 0]):.3f}")
print(f"The annual MPC in the {hank_lb.name} is {np.sum(M[:4, 0]):.3f}")

htm, whtm, phtm = compute_htm(ss)
print(f"The share of HtM is {htm:.3f}")
print(f"The share of wealthy HtM is {whtm:.3f}")
print(f"The share of poor HtM is {phtm:.3f}")

Liquid assets:  2.80
Illiquid assets:  11.20
Asset market clearing:  2.36e-06
Bond market clearing: -2.24e-08
Goods market clearing (untargeted):  6.48e-08
inv_res: 0.000000
val_res: 0.000000
nkpc_res: 0.000000
i_res: 0.000000
q_lb_res: 0.000000
rpost_res: 0.000000
B_res: -0.000000
equity_res: 0.000000
wnkpc_res: 0.000000
asset_mkt: 0.000002
bond_mkt: -0.000000
goods_mkt: 0.000000
The quarterly MPC in the Baseline Two-Asset HANK is 0.301
The annual MPC in the Baseline Two-Asset HANK is 0.396
The share of HtM is 0.410
The share of wealthy HtM is 0.384
The share of poor HtM is 0.026


In [6]:
ss.toplevel

{'beta': 0.988,
 'eis': 1.0,
 'rb': 0.0024906793143211203,
 'chi0': 3.781923472240388,
 'chi1': 37.56963403326428,
 'chi2': 2,
 'chi3': 0.0,
 'tax': 0.3,
 'w': 0.6585889824136807,
 'Transfer': 0.06427807925421791,
 'bmax': 20000,
 'amax': 20000,
 'kmax': 1,
 'nB': 50,
 'nA': 50,
 'nK': 50,
 'nZ': 3,
 'rho_z': 0.966,
 'sigma_z': 0.92,
 'Y': 1,
 'Z': 0.5458288090373042,
 'alpha': 0.27555211934495133,
 'ra_e': 0.00783354740858094,
 'delta': 0.02,
 'epsI': 4,
 'kappap': 0.1,
 'mup': 1.1,
 'pi_ss': 0.0049629315732038215,
 'tau_d': 0.4564452149049416,
 'Gbar': 0.2,
 'Bgbar': 2.8,
 'phi_G': 0.1,
 'kappaw': 0.1,
 'muw': 1.1,
 'vphi': 0.6151985150929595,
 'frisch': 1.0,
 'rbar': 0.0024906793143211203,
 'phi': 1.25,
 'pibar': 0.0049629315732038215,
 'rho_i': 0.8,
 'delta_lb': 0.95,
 'Bg': 2.8,
 'i': 0.007465971958532602,
 'Q': 1.0,
 'K': 9.0,
 'N': 1.0,
 'mc': 0.9090909090909091,
 'inv_res': 0.0,
 'val_res': 0.0,
 'pi': 0.004962931573203921,
 'nkpc_res': 0.0,
 'rb_e': 0.0024906793143211203,
 'i_

In [7]:
@sj.simple
def G_rule_ss(rb, tax, w, N, Transfer, Bg, T_d):
    T = tax * w * N + T_d
    G = T - Transfer - rb * Bg
    Bres = rb * Bg + (G + Transfer) - T
    Gbar = G
    Bgbar = Bg
    FD = 0  # fiscal deficit
    PD = Transfer + G - T  # primary deficit (does not include interest payments)
    return G, Bres, Gbar, Bgbar, FD, PD, T

def ssmap(ss, ra, rb, Y):
    """Find new steady state given Bg"""
    # evaluate production side
    capital_share = ss['alpha'] / ss['mup']
    K = (capital_share / (ra + ss['delta'])) * Y
    N = (Y / ss['Z'] / K ** ss['alpha']) ** (1 / (1 - ss['alpha']))
    labor_share = (1 - ss['alpha']) / ss['mup']
    w = labor_share * Y / N
    I = ss['delta'] * K
    div = (Y - w * N - I) * (1-ss['tau_d'])
    T_d =  (Y - w * N - I) * ss['tau_d']  
    
    p = div / ra  # (Y - w * N - r * K) / ra + (1 - delta/ra) * K

    # calculate new intercept for the fiscal rule
    d = {**ss, 'rb': rb, 'w': w, 'N': N, 'T_d': T_d}
    G = G_rule_ss.steady_state(d)['G']
    Bgbar = ss['Bg']
    Gbar = G
    
    # now evaluate household
    d = {**ss, 'N': N, 'w': w, 'rb': rb, 'ra': ra}
    out = hh_ext.steady_state(d)

    # create residuals
    asset_mkt = out['A'] + out['B'] - p - ss['Bg']
    bond_mkt = out['B'] - ss['Bg']
    labor_mkt = ss['vphi'] * N ** (1 / ss['frisch']) - (1 - ss['tax']) * w * out['UCE'] / ss['muw']

    # other things of interest
    #pshare = p / out['A']  # p / (p + ss['Bg'])
    Chi = out['CHI']
    goods_mkt = out['C'] + I + G + Chi - Y

    return asset_mkt, labor_mkt, bond_mkt, {**ss, **out, **dict(K=K, N=N, w=w, I=I, div=div, p=p, G=G, Gbar=Gbar, Bgbar=Bgbar,
                                                      Y=Y, ra_e=ra, i=(1+rb)*(1+ss['pi'])-1,
                                                      rb=rb, rb_e=rb, ra=ra, Chi=Chi, goods_mkt=goods_mkt,
                                                      rbar=rb)}

def hank_ss_Bg(dBg, ss):
    # Define new value of productivity
    Bgnew = ss['Bg'] + dBg * 4 * ss['Y']

    # Compute new equilibrium level of output and interest rate    
    ssalt = {**ss, 'Bg': Bgnew}
    (ra, rb, Y), _ = broyden_solver(lambda x: np.array(ssmap(ssalt, *x)[:3]), np.array([ss['ra'], ss['rb'], ss['Y']]))
    ssnew = ssmap(ssalt, ra, rb, Y)[3]

    # Check Walras law
    print(ssnew['goods_mkt'])

    return ssnew

In [8]:
# Compute the new steady state
d_Bg = 0.1
print(f"Compute the new steady state with d_Bg = {d_Bg:.3f}...")
c_new = hank_ss_Bg(d_Bg, ss)
ss_new = hank_lb.steady_state(c_new, dissolve=['G_rule', 'taylor_smooth'])
ss_hank_ini, ss_hank_end = ss, ss_new

Compute the new steady state with d_Bg = 0.100...
On iteration 0
x = 0.008, 0.002, 1.000
y = -0.400, -0.000, -0.400


On iteration 1
x = 0.008, 0.003, 0.997
y = -0.038, -0.000, 0.048


On iteration 2
x = 0.008, 0.003, 0.997
y = 0.010, 0.000, -0.011


On iteration 3
x = 0.008, 0.003, 0.997
y = -0.000, 0.000, -0.000


On iteration 4
x = 0.008, 0.003, 0.997
y = -0.000, -0.000, 0.000


On iteration 5
x = 0.008, 0.003, 0.997
y = -0.000, -0.000, 0.000


4.522694152520046e-08


In [9]:
for k, v in ss_hank_end.toplevel.items():
    if is_residual(k):
        print(f"{k}: {v:.6f}")

inv_res: 0.000000
val_res: -0.000000
nkpc_res: 0.000000
i_res: -0.000000
q_lb_res: -0.000000
rpost_res: 0.000000
B_res: 0.000000
equity_res: -0.000000
wnkpc_res: -0.000000
asset_mkt: -0.000000
bond_mkt: 0.000000
goods_mkt: 0.000000


## Table 4: comparison of SS

In [10]:
df = pd.DataFrame()

df['Variable'] = [
    'Bonds (% GDP)',
    'Illiquid real interest rate',
    'Liquid real interest rate',
    'Liquid nominal interest rate',
    'Output',
    'Investment',
    'Consumption',
    'Govt. consumption',
    'Portfolio costs',
    'Total Tax revenue',
    'Primary surplus (% GDP)'
]
df['Initial steady state'] = [
    100*ss_hank_ini['B']/4,
    100*((1+ss_hank_ini['ra'])**4-1),
    100*((1+ss_hank_ini['rb'])**4-1),
    100*((1+ss_hank_ini['i'])**4-1),
    100*ss_hank_ini['Y'],
    100*ss_hank_ini['I'],
    100*ss_hank_ini['C'],
    100*ss_hank_ini['G'],
    100*ss_hank_ini['CHI'],
    100*ss_hank_ini['T'],
    -100*ss_hank_ini['PD']/ss_hank_ini['Y'],
]

df['HANK'] = [
    100*ss_hank_end['B']/4,
    100*((1+ss_hank_end['ra'])**4-1),
    100*((1+ss_hank_end['rb'])**4-1),
    100*((1+ss_hank_end['i'])**4-1),
    100*ss_hank_end['Y'],
    100*ss_hank_end['I'],
    100*ss_hank_end['C'],
    100*ss_hank_end['G'],
    100*ss_hank_end['CHI'],
    100*ss_hank_end['T'],
    -100*ss_hank_end['PD']/ss_hank_end['Y'],
]

df['Difference HANK'] = df['HANK'] - df['Initial steady state']

s = df.to_latex(index=False, float_format="%.3f")
print(s)
with open(os.path.join('results', 'table_4.tex'), 'w') as tf:
     tf.write(s)

semi_elasticity = df.loc[2, 'Difference HANK']/np.log(8/7)
print(f"The semi-elasticity (dr*/dlog(Bg)) is {semi_elasticity:.2f} %.")

\begin{tabular}{lrrr}
\toprule
Variable & Initial steady state & HANK & Difference HANK \\
\midrule
Bonds (% GDP) & 70.000 & 80.000 & 10.000 \\
Illiquid real interest rate & 3.170 & 3.197 & 0.027 \\
Liquid real interest rate & 1.000 & 1.307 & 0.307 \\
Liquid nominal interest rate & 3.020 & 3.333 & 0.313 \\
Output & 100.000 & 99.742 & -0.258 \\
Investment & 18.000 & 17.911 & -0.089 \\
Consumption & 60.292 & 60.549 & 0.257 \\
Govt. consumption & 20.000 & 19.606 & -0.394 \\
Portfolio costs & 1.708 & 1.676 & -0.032 \\
Total Tax revenue & 27.125 & 27.075 & -0.051 \\
Primary surplus (% GDP) & 0.697 & 1.043 & 0.346 \\
\bottomrule
\end{tabular}

The semi-elasticity (dr*/dlog(Bg)) is 2.30 %.


In [11]:
# Serialize data into file:
json.dump(ss_hank_ini.toplevel, open(os.path.join("ss", "ss_hank_ini.json"), 'w'), indent=4)
json.dump(ss_hank_end.toplevel, open(os.path.join("ss", "ss_hank_end.json"), 'w'), indent=4)

## Compute the SS if the Central Bank does not change rbar and allows for a higher inflation target

In [12]:
# Linear calculation
delta_pi = (ss_hank_end['rb'] - ss_hank_ini['rbar'])/(ss_hank_ini['phi']-1)
print(f"In a linear approximation, the increase in annual inflation will be equal to {100*((1+delta_pi)**4-1):.2f} pp")
# Non-linear calculation
pi_new = (1+ss_hank_ini['pibar']) * ((1 + ss_hank_end['rb'])/(1 + ss_hank_ini['rbar'])) ** (1/(ss_hank_ini['phi']-1)) - 1
delta_pi_nl = pi_new - ss_hank_ini['pi']
print(f"In the non-linear model, the increase in annual inflation will be equal to {100*((1+delta_pi_nl)**4-1):.2f} pp")

ss_hank_end_candidate = ss_hank_end.copy()
ss_hank_end_candidate['rbar'] = ss_hank_ini['rbar']
ss_hank_end_candidate['pibar'] = ss_hank_ini['pibar']
for v in ['pi', 'pi_ss', 'piw']:
    ss_hank_end_candidate[v] = pi_new # ss_hank_ini[v] + delta_pi
ss_hank_end_candidate['i'] = (1+pi_new) * (1+ss_hank_end['rb']) - 1
ss_hank_end_no_reaction = hank_lb.steady_state(ss_hank_end_candidate, dissolve=['G_rule', 'taylor_smooth'])

In a linear approximation, the increase in annual inflation will be equal to 1.22 pp
In the non-linear model, the increase in annual inflation will be equal to 1.23 pp


In [13]:
def is_residual(name):
    if '_res' in name:
        return True
    elif '_mkt' in name:
        return True
    else:
        return False

for k, v in ss_hank_end_no_reaction.toplevel.items():
    if is_residual(k):
        print(f"{k}: {v:.6f}")

inv_res: 0.000000
val_res: -0.000000
nkpc_res: 0.000000
i_res: -0.000000
q_lb_res: -0.000000
rpost_res: 0.000000
B_res: 0.000000
equity_res: -0.000000
wnkpc_res: -0.000000
asset_mkt: -0.000000
bond_mkt: 0.000000
goods_mkt: 0.000000


In [14]:
json.dump(ss_hank_end_no_reaction.toplevel, open(os.path.join("ss", "ss_hank_end_no_reaction.json"), 'w'), indent=4)